# Collecte des données footballistiques

### Configuration de Selenium et Lancement du Site Web

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

website = "https://fbref.com/en/comps/9/2024-2025/2024-2025-Premier-League-Stats"
path = "../../data/chromedriver-win32/chromedriver.exe"
service = Service(path)
driver = webdriver.Chrome(service=service)

driver.get(website) 



### Extraction des Équipes et de leurs Liens depuis la Page Web

In [11]:
team_data = []
team_elements = driver.find_elements(By.CSS_SELECTOR,'td[data-stat="team"] > a')
seen = set()

for t in team_elements:
    name = t.text.strip()
    href = t.get_attribute('href')
    
    if name and name not in seen:
        team_data.append((name, href))
        seen.add(name)

print(team_data)

[('Liverpool', 'https://fbref.com/en/squads/822bd0ba/2024-2025/Liverpool-Stats'), ('Arsenal', 'https://fbref.com/en/squads/18bb7c10/2024-2025/Arsenal-Stats'), ('Manchester City', 'https://fbref.com/en/squads/b8fd03ef/2024-2025/Manchester-City-Stats'), ('Chelsea', 'https://fbref.com/en/squads/cff3d9bb/2024-2025/Chelsea-Stats'), ('Newcastle Utd', 'https://fbref.com/en/squads/b2b47a98/2024-2025/Newcastle-United-Stats'), ('Aston Villa', 'https://fbref.com/en/squads/8602292d/2024-2025/Aston-Villa-Stats'), ("Nott'ham Forest", 'https://fbref.com/en/squads/e4a775cb/2024-2025/Nottingham-Forest-Stats'), ('Brighton', 'https://fbref.com/en/squads/d07537b9/2024-2025/Brighton-and-Hove-Albion-Stats'), ('Bournemouth', 'https://fbref.com/en/squads/4ba7cbea/2024-2025/Bournemouth-Stats'), ('Brentford', 'https://fbref.com/en/squads/cd051869/2024-2025/Brentford-Stats'), ('Fulham', 'https://fbref.com/en/squads/fd962109/2024-2025/Fulham-Stats'), ('Crystal Palace', 'https://fbref.com/en/squads/47c64c55/2024-2

In [12]:
for i in team_data:
    print(i)

('Liverpool', 'https://fbref.com/en/squads/822bd0ba/2024-2025/Liverpool-Stats')
('Arsenal', 'https://fbref.com/en/squads/18bb7c10/2024-2025/Arsenal-Stats')
('Manchester City', 'https://fbref.com/en/squads/b8fd03ef/2024-2025/Manchester-City-Stats')
('Chelsea', 'https://fbref.com/en/squads/cff3d9bb/2024-2025/Chelsea-Stats')
('Newcastle Utd', 'https://fbref.com/en/squads/b2b47a98/2024-2025/Newcastle-United-Stats')
('Aston Villa', 'https://fbref.com/en/squads/8602292d/2024-2025/Aston-Villa-Stats')
("Nott'ham Forest", 'https://fbref.com/en/squads/e4a775cb/2024-2025/Nottingham-Forest-Stats')
('Brighton', 'https://fbref.com/en/squads/d07537b9/2024-2025/Brighton-and-Hove-Albion-Stats')
('Bournemouth', 'https://fbref.com/en/squads/4ba7cbea/2024-2025/Bournemouth-Stats')
('Brentford', 'https://fbref.com/en/squads/cd051869/2024-2025/Brentford-Stats')
('Fulham', 'https://fbref.com/en/squads/fd962109/2024-2025/Fulham-Stats')
('Crystal Palace', 'https://fbref.com/en/squads/47c64c55/2024-2025/Crystal-

### Chargement des Pages Équipes et Gestion des Timeouts

In [13]:
from selenium.common.exceptions import TimeoutException

for i in range(len(team_data)):
    team, href = team_data[i]
    print(f"Loading {team}...")
    driver.get(href)
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "table.stats_table"))
        )
    except TimeoutException:
        print(f"Timeout loading {team}, skipping...")
        continue

Loading Liverpool...
Loading Arsenal...
Loading Manchester City...
Loading Chelsea...
Loading Newcastle Utd...
Loading Aston Villa...
Loading Nott'ham Forest...
Loading Brighton...
Loading Bournemouth...
Loading Brentford...
Loading Fulham...
Loading Crystal Palace...
Loading Everton...
Loading West Ham...
Loading Manchester Utd...
Loading Wolves...
Loading Tottenham...
Loading Leicester City...
Loading Ipswich Town...
Loading Southampton...


### Extraction des Données des Joueurs de Toutes les Équipes et Création du DataFrame

In [14]:
import pandas as pd
from selenium.common.exceptions import TimeoutException

all_teams_players = []
for i in range(len(team_data)):
    team, href = team_data[i]
    print(f"Loading {team}...")
    driver.get(href)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "stats_standard_9"))
        )
        table = driver.find_element(By.ID, 'stats_standard_9')
        
        column_headers = table.find_elements(By.CSS_SELECTOR, "thead tr:nth-child(2) th")
        column_headers_text = [h.text for h in column_headers][:17]
        

        players_rows = table.find_elements(By.CSS_SELECTOR,"tbody tr")
        player_data = []
        for row in players_rows:
            player_name = row.find_element(By.TAG_NAME, "th").text
            cells = row.find_elements(By.TAG_NAME, "td")
            row_data = [player_name]+[cell.text for cell in cells[:16]]
            
            player_data.append(row_data)
            
         
        print("good")
        
        team_df = pd.DataFrame(player_data, columns=column_headers_text)
        team_df.insert(0, "Team", team)
        all_teams_players.append(team_df)
        
    except TimeoutException:
        print(f"Timeout loading {team}, skipping...")
        continue
    
players_All = pd.concat(all_teams_players,ignore_index=True)
print(players_All)

Loading Liverpool...
good
Loading Arsenal...
good
Loading Manchester City...
good
Loading Chelsea...
good
Loading Newcastle Utd...
good
Loading Aston Villa...
good
Loading Nott'ham Forest...
good
Loading Brighton...
good
Loading Bournemouth...
good
Loading Brentford...
good
Loading Fulham...
good
Loading Crystal Palace...
good
Loading Everton...
good
Loading West Ham...
good
Loading Manchester Utd...
good
Loading Wolves...
good
Loading Tottenham...
good
Loading Leicester City...
good
Loading Ipswich Town...
good
Loading Southampton...
good
            Team                 Player   Nation Pos Age  MP Starts    Min  \
0      Liverpool          Mohamed Salah   eg EGY  FW  32  38     38  3,371   
1      Liverpool        Virgil van Dijk   nl NED  DF  33  37     37  3,330   
2      Liverpool       Ryan Gravenberch   nl NED  MF  22  37     37  3,160   
3      Liverpool    Alexis Mac Allister   ar ARG  MF  25  35     30  2,599   
4      Liverpool        Ibrahima Konaté   fr FRA  DF  25  31    

In [15]:
players_All.head(100)

,Team,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG
0,Liverpool,Mohamed Salah,eg EGY,FW,32,38,38,"3,371",37.5,29,18,47,20,9,9,1,0,25.2
1,Liverpool,Virgil van Dijk,nl NED,DF,33,37,37,"3,330",37.0,3,1,4,3,0,0,5,0,2.2
2,Liverpool,Ryan Gravenberch,nl NED,MF,22,37,37,"3,160",35.1,0,4,4,0,0,0,6,1,1.1
3,Liverpool,Alexis Mac Allister,ar ARG,MF,25,35,30,"2,599",28.9,5,5,10,5,0,0,6,0,2.8
4,Liverpool,Ibrahima Konaté,fr FRA,DF,25,31,30,"2,560",28.4,1,2,3,1,0,0,5,0,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Manchester City,Abdukodir Khusanov,uz UZB,DF,20,6,6,503,5.6,0,0,0,0,0,0,1,0,0.0
96,Manchester City,James Mcatee,eng ENG,"MF,FW",21,15,3,349,3.9,3,0,3,3,0,0,1,0,2.8
97,Manchester City,Jahmai Simpson-Pusey,eng ENG,DF,18,2,1,96,1.1,0,0,0,0,0,0,1,0,0.0
98,Manchester City,Rodri,es ESP,MF,28,3,1,73,0.8,0,0,0,0,0,0,0,0,0.0


### Extraction des Données des Matchs de Toutes les Équipes et Création du DataFrame

In [16]:
all_teams_data2 = []
for i in range(len(team_data)):
    team, href = team_data[i]
    print(f"Loading matches of {team}...")
    
    driver.get(href)
    
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "matchlogs_for"))
        )
        
        table = driver.find_element(By.ID, "matchlogs_for")
        
        column_headers = table.find_elements(By.CSS_SELECTOR, "thead tr th")
        column_headers_text = [h.text for h in column_headers][:-2]
        
        matches_row = table.find_elements(By.CSS_SELECTOR, "tbody tr")
       
        matches_data = []
        for row in matches_row:
            matche_date = row.find_element(By.TAG_NAME, "th").text
            cells = row.find_elements(By.TAG_NAME, "td")
            row_data = [matche_date]+[cell.text for cell in cells[:-2]]
            
            matches_data.append(row_data)
        
        print(column_headers_text)
        
        match_df = pd.DataFrame(matches_data, columns=column_headers_text)
        match_df.insert(0,"Team",team)
        all_teams_data2.append(match_df)
        
    except TimeoutException:
        print(f"Timeout loading {team}, skipping...")
        continue
    
matches_All = pd.concat(all_teams_data2,ignore_index=True) 



Loading matches of Liverpool...
['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation', 'Opp Formation', 'Referee']
Loading matches of Arsenal...
['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation', 'Opp Formation', 'Referee']
Loading matches of Manchester City...
['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation', 'Opp Formation', 'Referee']
Loading matches of Chelsea...
['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation', 'Opp Formation', 'Referee']
Loading matches of Newcastle Utd...
['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation', 'Opp Fo

In [17]:
matches_All.head(100)

,Team,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee
0,Liverpool,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2,0,Ipswich Town,2.6,0.5,62,"30,014",Virgil van Dijk,4-2-3-1,4-2-3-1,Tim Robinson
1,Liverpool,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2,0,Brentford,2.5,0.5,62,"60,017",Virgil van Dijk,4-2-3-1,4-4-2,Stuart Attwell
2,Liverpool,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3,0,Manchester Utd,1.8,1.4,47,"73,738",Virgil van Dijk,4-2-3-1,4-2-3-1,Anthony Taylor
3,Liverpool,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0,1,Nott'ham Forest,0.9,0.4,68,"60,344",Virgil van Dijk,4-2-3-1,4-2-3-1,Michael Oliver
4,Liverpool,2024-09-17,21:00 (20:00),Champions Lg,League phase,Tue,Away,W,3,1,it Milan,3.1,0.6,51,"59,826",Virgil van Dijk,4-2-3-1,4-2-3-1,Espen Eskås
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Arsenal,2025-02-02,16:30 (17:30),Premier League,Matchweek 24,Sun,Home,W,5,1,Manchester City,1.0,0.8,46,"60,355",Martin Ødegaard,4-3-3,4-2-3-1,Peter Bankes
96,Arsenal,2025-02-05,20:00 (21:00),EFL Cup,Semi-finals,Wed,Away,L,0,2,Newcastle Utd,,,68,"52,173",Martin Ødegaard,4-3-3,5-4-1,Simon Hooper
97,Arsenal,2025-02-15,12:30 (13:30),Premier League,Matchweek 25,Sat,Away,W,2,0,Leicester City,1.4,0.2,60,"31,968",Martin Ødegaard,4-3-3,4-2-3-1,Samuel Barrott
98,Arsenal,2025-02-22,15:00 (16:00),Premier League,Matchweek 26,Sat,Home,L,0,1,West Ham,1.2,1.1,68,"60,262",Martin Ødegaard,4-3-3,5-3-2,Craig Pawson


In [18]:
players_All.to_csv('../../data/raw_data/players.csv', index=False)
matches_All.to_csv('../../data/raw_data/matches.csv', index=False)

players_All.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Team    754 non-null    object
 1   Player  754 non-null    object
 2   Nation  702 non-null    object
 3   Pos     702 non-null    object
 4   Age     702 non-null    object
 5   MP      702 non-null    object
 6   Starts  702 non-null    object
 7   Min     702 non-null    object
 8   90s     702 non-null    object
 9   Gls     702 non-null    object
 10  Ast     702 non-null    object
 11  G+A     702 non-null    object
 12  G-PK    702 non-null    object
 13  PK      702 non-null    object
 14  PKatt   702 non-null    object
 15  CrdY    702 non-null    object
 16  CrdR    702 non-null    object
 17  xG      702 non-null    object
dtypes: object(18)
memory usage: 106.2+ KB
